<a href="https://colab.research.google.com/github/tkchiang/Educational/blob/main/UV_Vis_for_Biomolecules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, HBox, VBox, interactive_output
import math

# **Defining models for scattering and absorbance of UV/Visible light**

Nanoparticles (such as proteins and BMC shells) can either scatter or absorb light. Physical/mathematical models have been developed that describe how they scatter and absorb, and are defined below:

## **Scattering:**

The following model is called the **Rayleigh Scattering model**, which approximates the intensity of scattering from a single spherical particle whose size is very small compared to the wavelength of the light being scattered (which is appropriate for our purposes, since our roughly spherical proteins are ~10—40 nm, and the wavelength of UV/Visible light is ~200—700 nm).

\begin{align}
S(r, \lambda) &= I_0 \,\frac{1 + \cos^2\theta}{2R^2} \left( \frac{2\pi}{\lambda} \right)^4 \left( \frac{n^2 - 1}{n^2 + 2} \right)^2 r^6 \\
&\propto \frac{r^6}{\lambda^4}
\end{align}

The **key takeaways** from the equation above are that the intensity of **scattered light** ($S$) scales with ($\propto$ = "proportional to") the **size of the particle to the 6th power** ($r^6$) and inversely with the **wavelength to the 4th power** ($1/\lambda^4$). Implicitly, we also know that a larger number of particles will also scatter more light, i.e. the intensity scales linearly with the **molar concentration** of scatterers.

Therefore:
 - **larger particles scatter more light** compared to smaller ones,
 - **light at shorter wavelengths is scattered more** than light at longer wavelengths, and
 - **higher concentrations of particles scatter more** than lower concentrations.

In [2]:
def ScatteringModel(lambda_, particleSize, Concentration):
  Scattering_Amplitude = Concentration*particleSize**6 # Scattering scales linearly with concentration and with size^6
  Scattering_vs_Lambda = Scattering_Amplitude / lambda_**4 # Scattering scales inversely with wavelength^4
  return Scattering_vs_Lambda

In [3]:
#@title **Simulated plots of Scattering intensity $S$ vs. wavelength $\lambda$ at different particle sizes $r$:**

def plotScattering(particleSize, Concentration):
    lambda_ = np.linspace(201, 700, 1000)
    s = ScatteringModel(lambda_, particleSize, Concentration)

    fig, ax = plt.subplots(1, 1, figsize=(8, 4))
    ax.plot(lambda_, s, label='Scattering')
    ax.set_xlabel('$\lambda$ (nm)')
    ax.set_ylabel('Intensity (a.u.)')
    ax.set_title('$S \propto C r^6 / \lambda^4$')
    ax.grid(True)
    ax.set_ylim([0,100])
    plt.show()

# Define sliders for interactive input
conc_slider = FloatSlider(value=30, min=0, max=100, step=0.1, description='Protein concentration (arbitrary units)',
                            layout={'width': '800px'}, style={'description_width': '400px'})
size_slider = FloatSlider(value=40, min=0.1, max=50, step=0.01, description='Average protein size (arbitrary units)',
                            layout={'width': '800px'}, style={'description_width': '400px'})

output = interactive_output(plotScattering, {
    'Concentration': conc_slider,
    'particleSize': size_slider,
})

ui = VBox([conc_slider, size_slider])
display(VBox([ui, output]))

## **Absorbance:**
The following model is the familiar **Beer-Lambert Law** that describes the intensity of light absorbance:

\begin{align}
A(\lambda) &= \epsilon(\lambda) \,l \,C
\end{align}

where $l$ is the optical path length and $C$ is the concentration. The extinction coefficient $\epsilon$ ("epsilon") relates the path length and concentration to the absorbance at some particular wavelength, measured as the Optical Density (OD). However, this extinction coefficient is not a constant, but rather it is a function of wavelength $\lambda$:

\begin{align}
\epsilon(\lambda) &\propto e^{-\frac{1}{2}\{(\lambda - \lambda_{\text{max}}) / \sigma)^2\}}
\end{align}

This function is known as the Gaussian curve, a.k.a the normal distribution, a.k.a. the bell curve. **$\lambda_{\text{max}}$ is the wavelength at which absorbance is the highest**, and $\sigma$ ("sigma") describes the width of the bell-shaped peak. For proteins, $\lambda_{\text{max}}$ is approximately 280 nm, which is a result of having aromatic residues such as tryptophan and tyrosine.

In [4]:
def AbsorbanceModel(lambda_, Concentration, Absorbance_Wavelength, Peak_Width):
  Absorbance_vs_Lambda = Concentration * np.exp(- (lambda_ - Absorbance_Wavelength)**2 / (2 * Peak_Width**2)) # Gaussian distribution describes absorbance vs wavelength
  return Absorbance_vs_Lambda

In [5]:
#@title **Simulated plot of Absorbance intensity $A$ vs. wavelength $\lambda$:**

def plotAbsorbance(Concentration, Absorbance_Wavelength, Peak_Width):
  r = np.linspace(10, 100, 1000)
  l = np.linspace(201, 700, 1000)
  a = AbsorbanceModel(l, Concentration, Absorbance_Wavelength, Peak_Width)

  fig, ax = plt.subplots(1, 1, figsize=(8, 4))
  ax.plot(l, a, '-', color='orange')
  ax.set_xlabel('$\lambda$')
  ax.set_ylabel('OD (A.U.)')
  ax.set_title(r'$A \propto C e^{-(\lambda - \lambda_{max})^2 / (2\sigma^2)}$')
  ax.grid(True)
  ax.set_xlim([200,500])
  ax.set_ylim([0,11])
  plt.show()

conc_slider = FloatSlider(value=5, min=0, max=10, step=0.1, description='Protein concentration (arbitrary units)',
                            layout={'width': '800px'}, style={'description_width': '400px'})
lmax_slider = FloatSlider(value=280, min=200, max=500, step=1, description='Absorbance maximum (nm)',
                            layout={'width': '800px'}, style={'description_width': '400px'})
sigma_slider = FloatSlider(value=15, min=1, max=50, step=1, description='Absorbance peak width (nm)',
                            layout={'width': '800px'}, style={'description_width': '400px'})

output = interactive_output(plotAbsorbance, {
    'Concentration': conc_slider,
    'Absorbance_Wavelength': lmax_slider,
    'Peak_Width': sigma_slider,
})

ui = VBox([conc_slider, lmax_slider,sigma_slider])
display(VBox([ui,output]))

# UV-Vis spectroscopy

In a typical UV-Vis measurement in which you want the measure protein concentration, the total extinction spectrum is measured and plotted, from which the OD at 280 nm is reported.

The measured extinction spectrum should look similar to the **red** curves shown below in the two plots. To help understand where the features of the curve arise from, the left plot shows a **dashed blue** line, which shows the contribution from the scattering, and the **dashed orange** and **dashed green** lines show the contributions from absorbance from the aromatic residues and peptide bonds, respectively. The sliders can be used to simulate what happens when the average particle size in the sample changes, the concentration changes, or the absorbance maximum changes (such as in protein samples that contain heme groups that absorb strongly at 410 nm).

See if you can fit the extinction curve to the real measured data in the plot on the right.

In [6]:
#@title **Simulated UV-Vis scattering, absorbance, and extinction spectra, and comparison to real data**

lambda_data = np.linspace(231,500,271)
spectrum = np.array([0.2948, 0.2658, 0.2418, 0.2150, 0.1892, 0.1634, 0.1432, 0.1180, 0.1026, 0.0870, 0.0746, 0.0654, 0.0578,
                     0.0524, 0.0480, 0.0434, 0.0422, 0.0402, 0.0380, 0.0358, 0.0362, 0.0356, 0.0358, 0.0360, 0.0362, 0.0372,
                     0.0382, 0.0392, 0.0428, 0.0436, 0.0464, 0.0472, 0.0482, 0.0508, 0.0526, 0.0550, 0.0584, 0.0600, 0.0628,
                     0.0636, 0.0652, 0.0668, 0.0676, 0.0686, 0.0694, 0.0702, 0.0714, 0.0712, 0.0696, 0.0672, 0.0666, 0.0626,
                     0.0592, 0.0570, 0.0546, 0.0514, 0.0480, 0.0466, 0.0452, 0.0428, 0.0394, 0.0388, 0.0364, 0.0348, 0.0334,
                     0.0310, 0.0302, 0.0286, 0.0280, 0.0272, 0.0256, 0.0258, 0.0232, 0.0234, 0.0226, 0.0218, 0.0210, 0.0204,
                     0.0194, 0.0196, 0.0188, 0.0180, 0.0182, 0.0174, 0.0158, 0.0158, 0.0158, 0.0160, 0.0152, 0.0152, 0.0144,
                     0.0144, 0.0136, 0.0138, 0.0128, 0.0138, 0.0130, 0.0122, 0.0132, 0.0124, 0.0124, 0.0124, 0.0116, 0.0126,
                     0.0118, 0.0118, 0.0118, 0.0118, 0.0110, 0.0110, 0.0120, 0.0112, 0.0112, 0.0112, 0.0112, 0.0104, 0.0114,
                     0.0114, 0.0106, 0.0106, 0.0106, 0.0108, 0.0098, 0.0108, 0.0116, 0.0108, 0.0108, 0.0100, 0.0100, 0.0100,
                     0.0100, 0.0102, 0.0102, 0.0102, 0.0094, 0.0094, 0.0092, 0.0094, 0.0094, 0.0094, 0.0094, 0.0094, 0.0094,
                     0.0094, 0.0086, 0.0086, 0.0086, 0.0096, 0.0096, 0.0096, 0.0088, 0.0088, 0.0088, 0.0088, 0.0088, 0.0088,
                     0.0088, 0.0080, 0.0080, 0.0080, 0.0080, 0.0080, 0.0080, 0.0090, 0.0080, 0.0082, 0.0082, 0.0082, 0.0080,
                     0.0082, 0.0082, 0.0082, 0.0082, 0.0082, 0.0082, 0.0082, 0.0074, 0.0074, 0.0074, 0.0074, 0.0074, 0.0074,
                     0.0074, 0.0066, 0.0074, 0.0066, 0.0066, 0.0066, 0.0066, 0.0074, 0.0066, 0.0066, 0.0076, 0.0068, 0.0076,
                     0.0068, 0.0068, 0.0068, 0.0068, 0.0060, 0.0068, 0.0058, 0.0068, 0.0068, 0.0068, 0.0068, 0.0068, 0.0060,
                     0.0060, 0.0068, 0.0060, 0.0060, 0.0060, 0.0060, 0.0052, 0.0060, 0.0052, 0.0052, 0.0062, 0.0052, 0.0052,
                     0.0052, 0.0052, 0.0052, 0.0054, 0.0052, 0.0052, 0.0044, 0.0052, 0.0052, 0.0052, 0.0052, 0.0044, 0.0044,
                     0.0044, 0.0054, 0.0044, 0.0054, 0.0052, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0054,
                     0.0054, 0.0054, 0.0044, 0.0046, 0.0054, 0.0054, 0.0044, 0.0046, 0.0046, 0.0046, 0.0046, 0.0046, 0.0046,
                     0.0046, 0.0046, 0.0046, 0.0046, 0.0046, 0.0046, 0.0046, 0.0046, 0.0046, 0.0046, 0.0046])

def plotExtinctionSpectrum(particleSize, Concentration, Absorbance_Wavelength):

    sigma_proteins = 10.7863 # Typical A280 absorbance peak width
    sigma_peptides = 15.3730 # Typical A190 peptide bond absorbance peak width
    lambda_peptides = 190 # Peptide bond absorbance wavelength
    lambda_range = np.linspace(200, 700, 1000)
    ScatteringOD = -np.log10(1 - ScatteringModel(lambda_range, particleSize, Concentration))
    AbsorbanceOD_residues = AbsorbanceModel(lambda_range, Concentration*1.7044e-04, Absorbance_Wavelength, sigma_proteins)
    AbsorbanceOD_peptidebonds = AbsorbanceModel(lambda_range, Concentration*0.0288, lambda_peptides, sigma_peptides)
    Extinction = ScatteringOD + AbsorbanceOD_residues + AbsorbanceOD_peptidebonds

    fig, axs = plt.subplots(1, 2, figsize=(18, 4))

    axs[0].plot(lambda_range, ScatteringOD, '--', label='Scattering $S$')
    axs[0].plot(lambda_range, AbsorbanceOD_residues, '--', label='Absorbance from residues $A_{res}$')
    axs[0].plot(lambda_range, AbsorbanceOD_peptidebonds, '--', label='Absorbance from peptide bonds $A_{pep}$')
    axs[0].plot(lambda_range, Extinction, 'r-', label='Extinction $E = S + A_{res} + A_{pep}$')
    axs[0].set_xlabel('Wavelength $\lambda$ (nm)')
    axs[0].set_ylabel('OD (A.U.)')
    axs[0].set_title('Simulated UV-Vis spectra')
    axs[0].legend()
    axs[0].grid(True)
    axs[0].set_ylim([-0.05, 0.3])
    axs[0].set_xlim([200, 400])

    axs[1].plot(lambda_data, spectrum, '.', label='Data (HT shells)', ms=2)
    axs[1].plot(lambda_range, Extinction, 'r-', label='Simulated')
    axs[1].set_xlabel('Wavelength $\lambda$ (nm)')
    axs[1].set_ylabel('OD (A.U.)')
    axs[1].set_title('Measured and Simulated spectra')
    axs[1].legend()
    axs[1].grid(True)
    axs[1].set_ylim([-0.05, 0.3])
    axs[1].set_xlim([200, 400])
    plt.show()



size_slider = FloatSlider(value=1e-1, min=0.1, max=15, step=0.01, description='Average protein size (arbitrary units)',
                            layout={'width': '800px'}, style={'description_width': '400px'})
conc_slider = FloatSlider(value=530, min=0, max=1000.0, step=0.1, description='Protein concentration (arbitrary units)',
                            layout={'width': '800px'}, style={'description_width': '400px'})
lmax_slider = FloatSlider(value=280, min=200, max=500, step=1, description='Absorbance maximum (nm)',
                            layout={'width': '800px'}, style={'description_width': '400px'})

output = interactive_output(plotExtinctionSpectrum, {
    'particleSize': size_slider,
    'Concentration': conc_slider,
    'Absorbance_Wavelength': lmax_slider,
})

ui = VBox([size_slider, conc_slider, lmax_slider])
display(VBox([ui,output]))